In [142]:
##Import libraries needed

import urllib.request
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse

In [143]:
## HTML content

page = urllib.request.urlopen('https://enb.iisd.org/enb/vol12/').read()
soup = BeautifulSoup(page)

In [126]:
## Function that extracts a date from a given string 

def extract_date(sdate):
    m = re.findall('\d{4}|\d{2}|January|February|March|April|May|June|July|August|September|October|November|December|\d{1}',sdate)
    month = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}
    if(len(m)==0):
        d=0
    if(len(m)==5):
        d = (int(m[0]),month[m[1]],int(m[len(m)-1]))
    if(len(m)==4):
        d = (int(m[0]),month[m[2]],int(m[len(m)-1]))
    if(len(m)==3):
        d = (int(m[0]),month[m[1]],int(m[2]))

    return d

In [127]:
## Function that extracts digit from a given string
def extract_number(sname):
    for i in sname.split():
        
        if i.isdigit():
            return int(i) 

In [128]:
## Function that helps to compose and order the list
def compute_list(list_string):
    list_cop = []

    for s in list_string:
        l = s.split('|')
      
        list_cop.append((extract_number(l[0]),extract_date(l[1]),l[2]))
        
    list_cop.sort(key=lambda a: a[0], reverse=False)

    return list_cop
    

In [129]:
## Function that extract the list of all the COPs from a webpage and return a list containing all the COPs with their number, date and place
def extract_list_COPs(soup):
    ## find all the different COPs (not named the same way)
    #Case 1 :"COP"+"\s"+"\d"+"\s"+"."+"\s" 
        #COP 1-9
    list = soup.find_all(string=re.compile("COP"+"\s"+"\d"+"\s"+"."+"\s"))

    #Case 2 : "COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"\d"
        #COP 10,23,24,25
    list += soup.find_all(string=re.compile("COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"\d"))
    
    #Case 3 :"COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"CMP"+"\s"+"\d"+"."+"."
        #COP 11,12,13,14,15,16,21,22
    list_2 = soup.find_all(string=re.compile("COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"CMP"+"\s"+"\d"+"."+"."))

    # Case 4 : "COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"CMP"+"\d"+"\s"
        #COP 17,18,19
    list_3 = soup.find_all(string=re.compile("COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"CMP"+"\d"+"\s"))
    
    ## Clean the lists to have all the same structure
    # Clean list_2
    for i in range(len(list_2)) :
        list_2[i] =  re.sub("- CMP"+"\s"+"."+".", '', list_2[i])


    # Clean list_3
    for i in range(len(list_3)) :
        list_3[i] =  re.sub("- CMP"+".", '', list_3[i])
    
    #combine all the lists
    list += list_2
    list += list_3
    
    return compute_list(list)


In [137]:
## Function that extract all the issues for each COPs and return a list in which for each COPs we have the issues corresponding with their number, date and html link 
def extract_details_COPS(soup):
    detail_COPs = []
    for row in soup.find_all("tr"):

        for col in row.find_all('td'):
        
            if("Issue" in col.string):
                a = row.find_previous_sibling('tr')
                b= a.find_next('th')
                detail = []
            
                if("</h3>COP" in str(b) and not("BIS" in str(b))):
                    date_td = col.find_next_sibling('td')
                
                    while("Summary" != date_td.string and "Summary and Analysis" != date_td.string):
                        ## extract issue number
                        issue = int(re.findall('\d+',col.string)[0])
                        
                        pdf_td = date_td.find_next_sibling('td')
                        ## extract html link
                        html_td = pdf_td.find_next_sibling('td')
                        html = html_td.find('a',href=True)['href']
                    
                        ## extract date
                        date = extract_date(date_td.string)    
                        
                        ##prepare for the next issue 
                        col = pdf_td.find_next('tr').find_next('td')
                        date_td = col.find_next_sibling('td')
                        
                        ##add the issue into the list
                        detail.append((issue,date,html))

                    detail_COPs.append(detail)
    return detail_COPs

In [ ]:
# Show how extract_list_COPS works
list_cops = extract_list_COPs(soup)

for cops_n in list_cops:
    print(cops_n)

In [ ]:
# Show how extract_details_COPs works
detail_cops = extract_details_COPS(soup)
k=1
for detail_n in detail_cops:
    print("COP",k)
    for j in detail_n:
        print(j)
        
    k+=1
